In [1]:
import os
import time
import random
import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o.model.regression import *
from h2o.model.binomial import *
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.feature_selection import RFE, RFECV
from sklearn import model_selection
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,10 hours 49 mins
H2O cluster timezone:,Asia/Taipei
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_ritakuo_jus54n
H2O cluster total nodes:,1
H2O cluster free memory:,1.457 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


## 一、匯入資料

In [2]:
df = pd.read_csv("train.csv", encoding = 'GB18030')
df.index = df["CUS_ID"]
index_df = df.index
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("CUS_ID", axis=1)
df.head()

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

## 二、資料整理

### 1. 將中文轉為英文

In [ ]:
def is_contains_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False

In [ ]:
names = df.columns
grade_range = {'低':'low', '中':'medium' , '中高': 'medium high', '高': 'high' }
for name in names:
    try :
        if is_contains_chinese(df[name].values):
            df[name] = df[name].map(grade_range)
    except:
        continue

### 2. 將 NaN 超過 79% 以上的刪除

In [14]:
df = df.drop("A_IND", axis=1)
df = df.drop("B_IND", axis=1)
df = df.drop("C_IND", axis=1)
df = df.drop("L1YR_C_CNT", axis=1)

### 3. 區分連續、類別變數

In [15]:
conti_vars = ["L1YR_A_ISSUE_CNT", "L1YR_B_ISSUE_CNT", "CHANNEL_A_POL_CNT", "CHANNEL_B_POL_CNT", "APC_CNT", \
              "INSD_CNT", "APC_1ST_YEARDIF", "ANNUAL_PREMIUM_AMT", "AG_CNT", "AG_NOW_CNT", "CLC_CUR_NUM", \
              "IM_CNT", "ANNUAL_INCOME_AMT", "BANK_NUMBER_CNT", "INSD_LAST_YEARDIF_CNT", "BMI", \
              "TERMINATION_RATE", "TOOL_VISIT_1YEAR_CNT", "DIEBENEFIT_AMT", "DIEACCIDENT_AMT", "POLICY_VALUE_AMT",\
             "ANNUITY_AMT", "EXPIRATION_AMT", "ACCIDENT_HOSPITAL_REC_AMT", "DISEASES_HOSPITAL_REC_AMT", \
             "OUTPATIENT_SURGERY_AMT", "INPATIENT_SURGERY_AMT", "PAY_LIMIT_MED_MISC_AMT", "FIRST_CANCER_AMT", \
             "ILL_ACCELERATION_AMT", "ILL_ADDITIONAL_AMT", "LONG_TERM_CARE_AMT", "MONTHLY_CARE_AMT", \
              "LIFE_INSD_CNT", "L1YR_GROSS_PRE_AMT"]

In [16]:
names = df.columns
categorical_vars = []
for index in range(names.shape[0]-1):
    categorical_vars.append(names[index])
for i in range(len(conti_vars)):
    categorical_vars.remove(conti_vars[i])
print("類別變數名稱：")
print(categorical_vars)

類別變數名稱：
['GENDER', 'AGE', 'CHARGE_CITY_CD', 'CONTACT_CITY_CD', 'EDUCATION_CD', 'MARRIAGE_CD', 'LAST_A_CCONTACT_DT', 'LAST_A_ISSUE_DT', 'LAST_B_ISSUE_DT', 'OCCUPATION_CLASS_CD', 'APC_1ST_AGE', 'INSD_1ST_AGE', 'IF_2ND_GEN_IND', 'RFM_R', 'REBUY_TIMES_CNT', 'LEVEL', 'RFM_M_LEVEL', 'LIFE_CNT', 'IF_ISSUE_A_IND', 'IF_ISSUE_B_IND', 'IF_ISSUE_C_IND', 'IF_ISSUE_D_IND', 'IF_ISSUE_E_IND', 'IF_ISSUE_F_IND', 'IF_ISSUE_G_IND', 'IF_ISSUE_H_IND', 'IF_ISSUE_I_IND', 'IF_ISSUE_J_IND', 'IF_ISSUE_K_IND', 'IF_ISSUE_L_IND', 'IF_ISSUE_M_IND', 'IF_ISSUE_N_IND', 'IF_ISSUE_O_IND', 'IF_ISSUE_P_IND', 'IF_ISSUE_Q_IND', 'IF_ADD_F_IND', 'IF_ADD_L_IND', 'IF_ADD_Q_IND', 'IF_ADD_G_IND', 'IF_ADD_R_IND', 'IF_ADD_IND', 'L1YR_PAYMENT_REMINDER_IND', 'L1YR_LAPSE_IND', 'LAST_B_CONTACT_DT', 'LAST_C_DT', 'IF_S_REAL_IND', 'IF_Y_REAL_IND', 'IM_IS_A_IND', 'IM_IS_B_IND', 'IM_IS_C_IND', 'IM_IS_D_IND', 'X_A_IND', 'X_B_IND', 'X_C_IND', 'X_D_IND', 'X_E_IND', 'X_F_IND', 'X_G_IND', 'X_H_IND', 'IF_HOUSEHOLD_CLAIM_IND', 'IF_ISSUE_INSD_A_IND'

In [17]:
binary_vars = []
for i in categorical_vars:
    if df[i].unique().shape[0] == 2 or df[i].unique().shape[0] == 3:
        if 'Y' in df[i].values or 'M' in df[i].values:
            binary_vars.append(i)
print("二元類別變數名稱：")
print(binary_vars)

二元類別變數名稱：
['GENDER', 'LAST_A_CCONTACT_DT', 'LAST_A_ISSUE_DT', 'LAST_B_ISSUE_DT', 'IF_2ND_GEN_IND', 'IF_ISSUE_A_IND', 'IF_ISSUE_B_IND', 'IF_ISSUE_C_IND', 'IF_ISSUE_D_IND', 'IF_ISSUE_E_IND', 'IF_ISSUE_F_IND', 'IF_ISSUE_G_IND', 'IF_ISSUE_H_IND', 'IF_ISSUE_I_IND', 'IF_ISSUE_J_IND', 'IF_ISSUE_K_IND', 'IF_ISSUE_L_IND', 'IF_ISSUE_M_IND', 'IF_ISSUE_N_IND', 'IF_ISSUE_O_IND', 'IF_ISSUE_P_IND', 'IF_ISSUE_Q_IND', 'IF_ADD_F_IND', 'IF_ADD_L_IND', 'IF_ADD_Q_IND', 'IF_ADD_G_IND', 'IF_ADD_R_IND', 'IF_ADD_IND', 'L1YR_PAYMENT_REMINDER_IND', 'L1YR_LAPSE_IND', 'LAST_B_CONTACT_DT', 'LAST_C_DT', 'IF_S_REAL_IND', 'IF_Y_REAL_IND', 'IM_IS_A_IND', 'IM_IS_B_IND', 'IM_IS_C_IND', 'IM_IS_D_IND', 'X_A_IND', 'X_B_IND', 'X_C_IND', 'X_D_IND', 'X_E_IND', 'X_F_IND', 'X_G_IND', 'X_H_IND', 'IF_HOUSEHOLD_CLAIM_IND', 'IF_ISSUE_INSD_A_IND', 'IF_ISSUE_INSD_B_IND', 'IF_ISSUE_INSD_C_IND', 'IF_ISSUE_INSD_D_IND', 'IF_ISSUE_INSD_E_IND', 'IF_ISSUE_INSD_F_IND', 'IF_ISSUE_INSD_G_IND', 'IF_ISSUE_INSD_H_IND', 'IF_ISSUE_INSD_I_IND', 'IF_I

In [18]:
multi_vars = []
for index in range(len(categorical_vars)):
    multi_vars.append(categorical_vars[index])
for i in range(len(binary_vars)):
    multi_vars.remove(binary_vars[i])
    
print("多元類別變數名稱：")
print((multi_vars))

多元類別變數名稱：
['AGE', 'CHARGE_CITY_CD', 'CONTACT_CITY_CD', 'EDUCATION_CD', 'MARRIAGE_CD', 'OCCUPATION_CLASS_CD', 'APC_1ST_AGE', 'INSD_1ST_AGE', 'RFM_R', 'REBUY_TIMES_CNT', 'LEVEL', 'RFM_M_LEVEL', 'LIFE_CNT', 'CUST_9_SEGMENTS_CD']


### 4. 將文字以數字表示

In [19]:
df = df.replace(["Y", "N"], [1, 0]) 
df = df.replace(["M", "F"], [1, 0])
df = df.replace(['low', 'medium high', 'medium', 'high'], [0, 1, 2, 3])
df = df.replace(['A1', 'A2', 'B1', 'B2', 'D', 'C1', 'C2', 'E'], [0, 1, 2, 3, 4, 5, 6, 7])
df = df.replace(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], [0, 1, 2, 3, 4, 5, 6, 7])
df.head()

,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,L1YR_B_ISSUE_CNT,...,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
CUS_ID,,,,,,,,,,,,,,,,,,,,,
3418,1.0,0,0,0,NaN,NaN,1,0,0,0,...,0.000174,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4302,1.0,0,0,0,NaN,NaN,1,0,0,0,...,0.008724,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5545,1.0,0,0,0,1.0,0.0,1,1,1,0,...,0.005359,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7207,1.0,0,0,0,NaN,0.0,1,0,0,0,...,0.000000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7213,1.0,0,0,0,1.0,0.0,0,0,0,0,...,0.000603,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### 5. 查看遺失值個數：遺失值過多，進行補值

In [ ]:
print('-'*10,'missing value summary','-'*10)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
for i in df:
    print(i)
    print(df[i].isnull().sum() / len(df))

## 三、用Generalized Low Rank Models (GLRM)做補值

### 1. 使用H2O

In [20]:
hf = h2o.H2OFrame(df)
hf.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:100000
Cols:127




,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,L1YR_B_ISSUE_CNT,LAST_B_ISSUE_DT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,APC_1ST_AGE,INSD_1ST_AGE,IF_2ND_GEN_IND,APC_1ST_YEARDIF,RFM_R,REBUY_TIMES_CNT,LEVEL,RFM_M_LEVEL,LIFE_CNT,IF_ISSUE_A_IND,IF_ISSUE_B_IND,IF_ISSUE_C_IND,IF_ISSUE_D_IND,IF_ISSUE_E_IND,IF_ISSUE_F_IND,IF_ISSUE_G_IND,IF_ISSUE_H_IND,IF_ISSUE_I_IND,IF_ISSUE_J_IND,IF_ISSUE_K_IND,IF_ISSUE_L_IND,IF_ISSUE_M_IND,IF_ISSUE_N_IND,IF_ISSUE_O_IND,IF_ISSUE_P_IND,IF_ISSUE_Q_IND,IF_ADD_F_IND,IF_ADD_L_IND,IF_ADD_Q_IND,IF_ADD_G_IND,IF_ADD_R_IND,IF_ADD_IND,ANNUAL_PREMIUM_AMT,AG_CNT,AG_NOW_CNT,CLC_CUR_NUM,ANNUAL_INCOME_AMT,L1YR_PAYMENT_REMINDER_IND,L1YR_LAPSE_IND,LAST_B_CONTACT_DT,LAST_C_DT,BANK_NUMBER_CNT,INSD_LAST_YEARDIF_CNT,BMI,IF_S_REAL_IND,IF_Y_REAL_IND,IM_CNT,IM_IS_A_IND,IM_IS_B_IND,IM_IS_C_IND,IM_IS_D_IND,TERMINATION_RATE,X_A_IND,X_B_IND,X_C_IND,X_D_IND,X_E_IND,X_F_IND,X_G_IND,X_H_IND,TOOL_VISIT_1YEAR_CNT,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,ACCIDENT_HOSPITAL_REC_AMT,DISEASES_HOSPITAL_REC_AMT,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,IF_HOUSEHOLD_CLAIM_IND,LIFE_INSD_CNT,IF_ISSUE_INSD_A_IND,IF_ISSUE_INSD_B_IND,IF_ISSUE_INSD_C_IND,IF_ISSUE_INSD_D_IND,IF_ISSUE_INSD_E_IND,IF_ISSUE_INSD_F_IND,IF_ISSUE_INSD_G_IND,IF_ISSUE_INSD_H_IND,IF_ISSUE_INSD_I_IND,IF_ISSUE_INSD_J_IND,IF_ISSUE_INSD_K_IND,IF_ISSUE_INSD_L_IND,IF_ISSUE_INSD_M_IND,IF_ISSUE_INSD_N_IND,IF_ISSUE_INSD_O_IND,IF_ISSUE_INSD_P_IND,IF_ISSUE_INSD_Q_IND,IF_ADD_INSD_F_IND,IF_ADD_INSD_L_IND,IF_ADD_INSD_Q_IND,IF_ADD_INSD_G_IND,IF_ADD_INSD_R_IND,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
type,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,real,int,int,int,int,real,real,real,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.46691905716040555,1.4773899999999989,2.205690000000001,2.3616400000000013,2.16907525365694,0.3152885962911059,0.35405,0.11817000000000004,0.11369,0.0033300000000000005,0.00547,1.6277299999999986,0.03108,1.3049666805497704,0.6810699999999994,0.2836199999999997,1.38934729715434,1.5056246180969457,0.54315,0.25876859845935973,1.34160406306211,1.0938855389823352,3.5579151600670276,5.644909905144756,1.06569,0.01877,0.00979,0.03589,0.0718,0.00138,0.02899,0.02675,0.00034,0.05627,0.1097,0.00255,0.00534,0.001,0.15122,0.011649999999999997,0.15022,0.18408,0.1138,0.18463,0.18946,0.00596,0.14625,0.24758,0.001234560769564638,0.9734,0.4566699999999996,0.55733,0.0006068425456175263,0.03528,0.08977,0.19188,0.17701,0.056027499999999945,0.1724875816240972,0.26424089736668477,0.31373,0.17888,0.38377999999999973,0.00634,0.12481,0.07316,0.17947,12.090984696216356,0.012564774614353454,0.3228026650127048,0.32856485464476504,0.08393189411976551,0.20137652307876994,0.0023008743322462536,0.0020207678917988836,0.23

### 2. 以 0.7 / 0.3 切割 Training / Validation Data

In [21]:
hf_train, hf_vali = hf.split_frame(ratios = [.69975], seed = 1234)
hf_y_train = hf_train["Y1"]
hf_y_vali = hf_vali["Y1"]
hf_train = hf_train[:-1]
hf_vali = hf_vali[:-1]
print(hf_train.shape)
print(hf_vali.shape)
print(hf_y_train.shape)
print(hf_y_vali.shape)

(70000, 126)
(30000, 126)
(70000, 1)
(30000, 1)


### 3. 將 Training / Validation Data 變數型態改為 factor 及 numeric

In [23]:
column_list = hf_train.col_names
for i in column_list:
    if i in categorical_vars:
        hf_train[i] = hf_train[i].asfactor()
    else:
        hf_train[i] = hf_train[i].asnumeric()
hf_train.describe()

Rows:70000
Cols:126




,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,L1YR_B_ISSUE_CNT,LAST_B_ISSUE_DT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,APC_1ST_AGE,INSD_1ST_AGE,IF_2ND_GEN_IND,APC_1ST_YEARDIF,RFM_R,REBUY_TIMES_CNT,LEVEL,RFM_M_LEVEL,LIFE_CNT,IF_ISSUE_A_IND,IF_ISSUE_B_IND,IF_ISSUE_C_IND,IF_ISSUE_D_IND,IF_ISSUE_E_IND,IF_ISSUE_F_IND,IF_ISSUE_G_IND,IF_ISSUE_H_IND,IF_ISSUE_I_IND,IF_ISSUE_J_IND,IF_ISSUE_K_IND,IF_ISSUE_L_IND,IF_ISSUE_M_IND,IF_ISSUE_N_IND,IF_ISSUE_O_IND,IF_ISSUE_P_IND,IF_ISSUE_Q_IND,IF_ADD_F_IND,IF_ADD_L_IND,IF_ADD_Q_IND,IF_ADD_G_IND,IF_ADD_R_IND,IF_ADD_IND,ANNUAL_PREMIUM_AMT,AG_CNT,AG_NOW_CNT,CLC_CUR_NUM,ANNUAL_INCOME_AMT,L1YR_PAYMENT_REMINDER_IND,L1YR_LAPSE_IND,LAST_B_CONTACT_DT,LAST_C_DT,BANK_NUMBER_CNT,INSD_LAST_YEARDIF_CNT,BMI,IF_S_REAL_IND,IF_Y_REAL_IND,IM_CNT,IM_IS_A_IND,IM_IS_B_IND,IM_IS_C_IND,IM_IS_D_IND,TERMINATION_RATE,X_A_IND,X_B_IND,X_C_IND,X_D_IND,X_E_IND,X_F_IND,X_G_IND,X_H_IND,TOOL_VISIT_1YEAR_CNT,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,ACCIDENT_HOSPITAL_REC_AMT,DISEASES_HOSPITAL_REC_AMT,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,IF_HOUSEHOLD_CLAIM_IND,LIFE_INSD_CNT,IF_ISSUE_INSD_A_IND,IF_ISSUE_INSD_B_IND,IF_ISSUE_INSD_C_IND,IF_ISSUE_INSD_D_IND,IF_ISSUE_INSD_E_IND,IF_ISSUE_INSD_F_IND,IF_ISSUE_INSD_G_IND,IF_ISSUE_INSD_H_IND,IF_ISSUE_INSD_I_IND,IF_ISSUE_INSD_J_IND,IF_ISSUE_INSD_K_IND,IF_ISSUE_INSD_L_IND,IF_ISSUE_INSD_M_IND,IF_ISSUE_INSD_N_IND,IF_ISSUE_INSD_O_IND,IF_ISSUE_INSD_P_IND,IF_ISSUE_INSD_Q_IND,IF_ADD_INSD_F_IND,IF_ADD_INSD_L_IND,IF_ADD_INSD_Q_IND,IF_ADD_INSD_G_IND,IF_ADD_INSD_R_IND,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G
type,enum,enum,enum,enum,enum,enum,enum,int,enum,int,enum,int,int,enum,int,int,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,int,int,int,real,enum,enum,enum,enum,real,real,real,enum,enum,int,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum
mins,,,,,,,,0.0,,0.0,,0.0,0.0,,0.0,0.0,,,,0.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,,,,,0.0,0.0,0.0,,,0.0,,,,,0.0,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,,,,,,,,,,,,,,,,,,,,,,,,0.0,,,,,,,,
mean,,,,,,,,0.11939999999999995,,0.0034000000000000002,,1.632728571428573,0.030842857142857154,,0.6814142857142863,0.28264285714285714,,,,0.2583607848672618,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.001233532495925406,0.9730142857142864,0.45731428571428606,0.5573428571428569,0.0006139072006350699,,,,,0.05593571428571424,0.17260730606491698,0.26406284794189705,,,0.3845428571428572,,,,,11.998725399019731,,,,,,,,,1.4058428571428558,0.0030825501746231643,0.004413054003447595,0.0008736904919162499,0.0023730939177456817,0.000307829398992483,0.151833740422685,0.13687258270955177,0.09025593114457474,0.058733128787260426,0.0946178027470581,0.014265378962867474,0.00426711042360974,0.0022231744578692578,0.0023696652904285874,0.019110723389468308,,0.03163125000000003,,,,,,,,,,,,,,,,,,,,,,,,0.0005305485281999996,,,,,,,,
maxs,,,,,,,,16.0,,4.0,,93.0,14.0,,4.0,9.0,,,,0.9302326,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.3788604,18.0,10.0,7.0,0.25,,,,,0.875,1.0,0.75,,,4.0,,,,,100.0,,,,,,,,,112.0,0.3622524,0.3622524,0.3785324,0.3773450999999999,0.2044154,0.8079999999999999,0.8064515999999999,0.952381,0.8242424,1.0,0.7345911999999999,0.666666667,0.3,0.8095238,0.7302405,,0.921875,,,,,,,,,,,,,,,,,,,,,,,,0.3278146,,,,,,,,
sigma,,

In [25]:
column_list = hf_vali.col_names
for i in column_list:
    if i in categorical_vars:
        hf_vali[i] = hf_vali[i].asfactor()
    else:
        hf_vali[i] = hf_vali[i].asnumeric()
hf_vali.describe()

Rows:30000
Cols:126




,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,L1YR_B_ISSUE_CNT,LAST_B_ISSUE_DT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,APC_1ST_AGE,INSD_1ST_AGE,IF_2ND_GEN_IND,APC_1ST_YEARDIF,RFM_R,REBUY_TIMES_CNT,LEVEL,RFM_M_LEVEL,LIFE_CNT,IF_ISSUE_A_IND,IF_ISSUE_B_IND,IF_ISSUE_C_IND,IF_ISSUE_D_IND,IF_ISSUE_E_IND,IF_ISSUE_F_IND,IF_ISSUE_G_IND,IF_ISSUE_H_IND,IF_ISSUE_I_IND,IF_ISSUE_J_IND,IF_ISSUE_K_IND,IF_ISSUE_L_IND,IF_ISSUE_M_IND,IF_ISSUE_N_IND,IF_ISSUE_O_IND,IF_ISSUE_P_IND,IF_ISSUE_Q_IND,IF_ADD_F_IND,IF_ADD_L_IND,IF_ADD_Q_IND,IF_ADD_G_IND,IF_ADD_R_IND,IF_ADD_IND,ANNUAL_PREMIUM_AMT,AG_CNT,AG_NOW_CNT,CLC_CUR_NUM,ANNUAL_INCOME_AMT,L1YR_PAYMENT_REMINDER_IND,L1YR_LAPSE_IND,LAST_B_CONTACT_DT,LAST_C_DT,BANK_NUMBER_CNT,INSD_LAST_YEARDIF_CNT,BMI,IF_S_REAL_IND,IF_Y_REAL_IND,IM_CNT,IM_IS_A_IND,IM_IS_B_IND,IM_IS_C_IND,IM_IS_D_IND,TERMINATION_RATE,X_A_IND,X_B_IND,X_C_IND,X_D_IND,X_E_IND,X_F_IND,X_G_IND,X_H_IND,TOOL_VISIT_1YEAR_CNT,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,ACCIDENT_HOSPITAL_REC_AMT,DISEASES_HOSPITAL_REC_AMT,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,IF_HOUSEHOLD_CLAIM_IND,LIFE_INSD_CNT,IF_ISSUE_INSD_A_IND,IF_ISSUE_INSD_B_IND,IF_ISSUE_INSD_C_IND,IF_ISSUE_INSD_D_IND,IF_ISSUE_INSD_E_IND,IF_ISSUE_INSD_F_IND,IF_ISSUE_INSD_G_IND,IF_ISSUE_INSD_H_IND,IF_ISSUE_INSD_I_IND,IF_ISSUE_INSD_J_IND,IF_ISSUE_INSD_K_IND,IF_ISSUE_INSD_L_IND,IF_ISSUE_INSD_M_IND,IF_ISSUE_INSD_N_IND,IF_ISSUE_INSD_O_IND,IF_ISSUE_INSD_P_IND,IF_ISSUE_INSD_Q_IND,IF_ADD_INSD_F_IND,IF_ADD_INSD_L_IND,IF_ADD_INSD_Q_IND,IF_ADD_INSD_G_IND,IF_ADD_INSD_R_IND,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G
type,enum,enum,enum,enum,enum,enum,enum,int,enum,int,enum,int,int,enum,int,int,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,int,int,int,real,enum,enum,enum,enum,real,real,real,enum,enum,int,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum
mins,,,,,,,,0.0,,0.0,,0.0,0.0,,0.0,0.0,,,,0.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,,,,,0.0,0.0,0.0,,,0.0,,,,,0.0,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,,,,,,,,,,,,,,,,,,,,,,,,0.0,,,,,,,,
mean,,,,,,,,0.11530000000000001,,0.0031666666666666666,,1.6160666666666659,0.03163333333333334,,0.680266666666667,0.28590000000000004,,,,0.25972677856694043,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.0012369890674006447,0.9743000000000004,0.45516666666666666,0.5572999999999999,0.0005904154066943777,,,,,0.05624166666666666,0.17220830852852617,0.2646570707879565,,,0.38199999999999995,,,,,12.307752908521824,,,,,,,,,1.4167333333333323,0.0030720368671564545,0.004377371704713056,0.0008548746176508638,0.002326608546668516,0.0003515623232603271,0.1511290775713889,0.13616454726323812,0.08913817805068848,0.05902862516731355,0.09413351353848995,0.014329183001894463,0.0043224471366324745,0.0022416351284539325,0.0024495121610756867,0.0184443722341743,,0.031104166666666648,,,,,,,,,,,,,,,,,,,,,,,,0.0005133965867333333,,,,,,,,
maxs,,,,,,,,22.0,,2.0,,97.0,17.0,,4.0,19.0,,,,0.8372093,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1523081,18.0,8.0,8.0,0.04166666,,,,,0.75,0.9210526,0.925,,,4.0,,,,,100.0,,,,,,,,,176.0,0.1170767,0.13256529999999994,0.2385341,0.3626428,0.2875879000000001,0.8,0.8602151,0.6825397,0.7272727,0.9375,0.6289308000000001,0.5128205,0.3,0.6428571,0.8591065000000001,,0.875,,,,,,,,,,,,,,,,,,,,,,,,0.1380289,,,,,,,

### 4. 利用 Validation Data 進行 GLRM 的調參

In [ ]:
glrm_params = {
                'k': [3, 4, 5, 6, 7, 8]
                'gamma_x': [0.0, 0.25, 0.5, 0.75, 1.0],               
                'gamma_y': [0.0, 0.25, 0.5, 0.75, 1.0]
                }

In [ ]:
glrm_grid = H2OGridSearch(
    model=H2OGeneralizedLowRankEstimator(loss="Quadratic", regularization_x="L2", regularization_y="L1", 
                                         max_iterations=10, multi_loss='Categorical', init_step_size=1e-3,
                                         min_step_size=1e-6),
    grid_id='glrm_grid1',                  
    hyper_params=glrm_params,             
    search_criteria={"strategy": "Cartesian"})

glrm_grid.train(training_frame=hf_vali)

In [ ]:
glrm_gridperf = glrm_grid.get_grid(sort_by='RMSE', decreasing = False)
glrm_gridperf

In [ ]:
best_glrm_model = glrm_gridperf.models[0]
best_glrm_model

### 5. Training Data 進行 GLRM 訓練

In [ ]:
k = 3, gamma_x = 0.001, gamma_y = 0.001
glrm_model = H2OGeneralizedLowRankEstimator(k=k, loss="Quadratic", regularization_x="L2", regularization_y="L1", max_iterations=10,
                                      gamma_x=gamma_x, gamma_y=gamma_y, multi_loss='Categorical', init_step_size=1e-3, min_step_size=1e-6)
glrm_model.train(training_frame=hf_train)
glrm_model.show()

In [ ]:
model_score = glrm_model.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
print(model_score)
plt.plot(model_score["iterations"], model_score["objective"])
plt.show()

### 6. 將整數連續型變數中為負的補0

In [ ]:
var_integer = ["L1YR_A_ISSUE_CNT", "L1YR_B_ISSUE_CNT", "CHANNEL_A_POL_CNT", "CHANNEL_B_POL_CNT", "APC_CNT", \
              "INSD_CNT", "AG_CNT", "AG_NOW_CNT", "CLC_CUR_NUM", "IM_CNT", "TOOL_VISIT_1YEAR_CNT"]

def relu(data):
    for i in data.columns:
        if i in var_integer:
            data[data[i]<0, i]=0
    return(data)

In [ ]:
hf_train_hat = glrm_model.predict(hf_train)
hf_train_hat = relu(hf_train_hat)
hf_train_hat

### 7. 利用 Validation Data 驗證補值結果：

### 將 Validation Data 取出無 NaN 的部分，自行塞入 50% 的 NaN 驗證 GLRM 模型是否準確

In [ ]:
hf_vali_real = hf_vali.na_omit()
hf_vali_real.head()

In [ ]:
hf_vali_na = h2o.deep_copy(hf_vali_real, 'vali_na')
hf_vali_na = hf_vali_na.insert_missing_values(0.5)
hf_vali_na.head()

In [ ]:
column_list = hf_vali_na.col_names
for i in column_list:
    if i in categorical_vars:
        hf_vali_na[i] = hf_vali_na[i].asfactor()
    else:
        hf_vali_na[i] = hf_vali_na[i].asnumeric()

In [ ]:
hf_vali_hat = glrm_model.predict(hf_vali_na)
hf_vali_hat = relu(hf_vali_hat)
hf_vali_hat

In [ ]:
def makelabel(data, dfna, name):
    k = h2o.as_list(data).values.reshape(-1)
    k1 = np.isnan(h2o.as_list(dfna).values.reshape(-1))
    k = h2o.H2OFrame(k[k1])
    k.names = [name]
    return k

In [ ]:
from h2o.model.regression import *

label = makelabel(hf_vali_real, hf_vali_na, 'label')
predict = makelabel(hf_vali_hat, hf_vali_na, 'predict')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(h2o.as_list(predict), h2o.as_list(label), alpha=0.3)
ax.plot(h2o.as_list(predict), h2o.as_list(predict), c='red')
display(fig)

### 查看補值結果

In [ ]:
df_vali_real = hf_vali_real.as_data_frame(use_pandas = True)
df_vali_na = hf_vali_na.as_data_frame(use_pandas = True)
df_vali_hat = hf_vali_hat.as_data_frame(use_pandas = True)

In [ ]:
df_vali_real = df_vali_real.astype(np.float32)
df_vali_na = df_vali_na.astype(np.float32)
df_vali_hat = df_vali_hat.astype(np.float32)

vali_na_newpre_df = df_vali_na.copy()
vali_na_newpre_df[df_vali_na.isna()] = df_vali_hat[df_vali_na.isna()].values

In [ ]:
contin_no_seletor = []
factor_no_select = []
for var in df_vali_real.columns:
    if var in conti_var:
        mse = mean_squared_error(df_vali_real[var], vali_na_newpre_df[var])
        if mse > 1:
            contin_no_seletor.append(var)
    if var in categorical_vars:
        acc = accuracy_score(df_vali_real[var], vali_na_newpre_df[var])        
        if acc < 0.85:
            factor_no_select.append(var)
            print('factor_vars', var,acc)

### 補值結果差的不做補值

### Training Data

In [ ]:
train_glrm = hf_train.as_data_frame(use_pandas=True)
train_hat = hf_train_hat.as_data_frame(use_pandas=True)

train_glrm = train_glrm.astype(np.float32)
train_hat = train_hat.astype(np.float32)

train_glrm_new = train_glrm.copy()
train_glrm_new[train_glrm.isna()] = train_hat[train_glrm.isna()].values

for i in contin_no_seletor + factor_no_select:
    train_glrm_new[train_glrm[i].isna()] = train_glrm[train_glrm[i].isna()].values

### Validation Data 

In [ ]:
for i in contin_no_seletor + factor_no_select:
    vali_na_newpre_df[df_vali_na[i].isna()] = df_vali_na[df_vali_na[i].isna()].values

## 四、利用 REFCV 進行 Feature Selection

### GLRM 補完值 Data 做 Feature Selection

In [ ]:
estimator = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc')
selector = RFE(estimator=estimator, n_features_to_select=30)
X_t_glrm = selector.fit_transform(train_glrm, y_train)
X_train_glrm, X_test_glrm, y_train_glrm, y_test_glrm = model_selection.train_test_split(X_t_glrm, y_train,
                                                                            test_size=0.25, random_state=0)

In [ ]:
clf_t_glrm = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc')

clf_t_glrm.fit(X_train_glrm, y_train_glrm)
y_pred = clf_t_glrm.predict(X_train_glrm)
fpr, tpr, thresholds = metrics.roc_curve(y_train_glrm, y_pred, pos_label=1)
auc = metrics.auc(fpr, tpr)
print('after auc = ', auc)

y_pred = clf_t_glrm.predict(X_test_glrm)
fpr, tpr, thresholds = metrics.roc_curve(y_test_glrm, y_pred, pos_label=1)
auc = metrics.auc(fpr, tpr)
print('after test auc = ', auc)

In [ ]:
features = X.columns[selector.get_support()]
print('number:', len(features))
print(features)

### Training Data 只取重要的變數

In [ ]:
var_names = list(train_glrm.columns)
for var in var_names:
    if var not in features:
        train_glrm = train_glrm.drop(var, axis=1)
train_glrm.head()

### Validation Data 只取重要的變數

In [ ]:
var_names = list(vali_na_newpre_df.columns)
for var in var_names:
    if var not in features:
        vali_na_newpre_df = vali_na_newpre_df.drop(var, axis=1)
vali_na_newpre_df.head()

## 五、利用 XGBoost 進行訓練及預測

### 1. Validation Data 使 XGBoost 自行補值，進行 XGBoost 調參

In [ ]:
parameters = {'max_depth' : range(3, 10, 2), 'min_child_weight' : range(1, 6, 2)}
 
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc')
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(x_vali, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {'gamma' : [i / 10.0 for i in range(0, 5)]}
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3,
                        min_child_weight=5)
 
clf = GridSearchCV(model, parameters,n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(x_vali, y_vali)

clf.best_params_, clf.best_score_

In [ ]:
parameters = {'subsample' : [i / 10.0 for i in range(6, 10)], 'colsample_bytree' : [i / 10.0 for i in range(6, 10)]} 
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, 
                         min_child_weight=5, gamma=0.1)
 
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(x_vali, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {
 'reg_alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100, 1000],
 'reg_lambda' : [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100, 1000]
}
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, min_child_weight=5, gamma = 0.1,
                         colsample_bytree=0.8, subsample=0.7)
clf = GridSearchCV(model, parameters, n_jobs=-1, cv = 5, scoring='roc_auc', verbose=0)
clf.fit(x_vali, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {
 'reg_alpha' : list(np.linspace(1e-3, 1e-1, 10)),
 'reg_lambda' : list(np.linspace(1e-5, 1e-3, 10))
}

model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, min_child_weight=5, gamma=0.1,
                         colsample_bytree=0.8, subsample=0.7)
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(x_vali, y_vali)
 
clf.best_params_, clf.best_score_

### 2. Training Data 使 XGBoost 自行補值，利用調參結果進行訓練

In [ ]:
xgbc = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=5, min_child_weight=1, gamma=0.1,
                         colsample_bytree=0.8, subsample=0.8, reg_alpha=0.077, reg_lambda=0.00083)
xgbc.fit(x_train, y_train)

In [ ]:
y_pred_train = xgbc.predict(x_train)
print(y_pred_train)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_train, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
y_pred_vali = xgbc.predict(x_vali)
fpr, tpr, thresholds = metrics.roc_curve(y_vali, y_pred_vali, pos_label=1)
metrics.auc(fpr, tpr)

### 3. Validation Data 利用 GLRM 補值，進行 XGBoost 調參

In [ ]:
parameters = {'max_depth' : range(3, 10, 2), 'min_child_weight' : range(1, 6, 2)}
 
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc')
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(vali_na_newpre_df, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {'gamma' : [i / 10.0 for i in range(0, 5)]}
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3,
                        min_child_weight=5)
 
clf = GridSearchCV(model, parameters,n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(vali_na_newpre_df, y_vali)

clf.best_params_, clf.best_score_

In [ ]:
parameters = {'subsample' : [i / 10.0 for i in range(6, 10)], 'colsample_bytree' : [i / 10.0 for i in range(6, 10)]} 
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, 
                         min_child_weight=5, gamma=0.1)
 
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(vali_na_newpre_df, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {
 'reg_alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100, 1000],
 'reg_lambda' : [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100, 1000]
}
model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, min_child_weight=5, gamma = 0.1,
                         colsample_bytree=0.8, subsample=0.7)
clf = GridSearchCV(model, parameters, n_jobs=-1, cv = 5, scoring='roc_auc', verbose=0)
clf.fit(vali_na_newpre_df, y_vali)
 
clf.best_params_, clf.best_score_

In [ ]:
parameters = {
 'reg_alpha' : list(np.linspace(1e-3, 1e-1, 10)),
 'reg_lambda' : list(np.linspace(1e-5, 1e-3, 10))
}

model = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=3, min_child_weight=5, gamma=0.1,
                         colsample_bytree=0.8, subsample=0.7)
clf = GridSearchCV(model, parameters, n_jobs=-1, cv=5, scoring='roc_auc', verbose=0)
clf.fit(vali_na_newpre_df, y_vali)
 
clf.best_params_, clf.best_score_

### 4. Training Data 利用 GLRM 補值，利用調參結果進行訓練

In [ ]:
xgbc_glrm = xgb.XGBRegressor(objective="reg:logistic", random_state=42, eval_metric='auc', max_depth=5, min_child_weight=1, gamma=0.1,
                         colsample_bytree=0.8, subsample=0.8, reg_alpha=0.077, reg_lambda=0.00083)
xgbc_glrm.fit(train_glrm, y_train)

In [ ]:
y_pred_train_glrm = xgbc_glrm.predict(train_glrm)
print(y_pred_train)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred_train_glrm, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
y_pred_vali_glrm = xgbc_glrm.predict(vali_na_newpre_df)
fpr, tpr, thresholds = metrics.roc_curve(y_vali, y_pred_vali_glrm, pos_label=1)
metrics.auc(fpr, tpr)

## 六、帶入Testing Data

In [ ]:
df_test = pd.read_csv("test.csv", encoding = 'GB18030')
ID = df_test["CUS_ID"]
df_test.index = df_test["CUS_ID"]
df_test = df_test.drop("CUS_ID", axis=1)
df_test.head()

### 1. 資料整理

In [ ]:
names = df_test.columns
grade_range = {'低':'low', '中':'medium' , '中高': 'medium high', '高': 'high' }
for name in names:
    try :
        if is_contains_chinese(df_test[name].values):
            df_test[name] = df_test[name].map(grade_range)
    except:
        continue

In [ ]:
df_test = df_test.drop("A_IND", axis=1)
df_test = df_test.drop("B_IND", axis=1)
df_test = df_test.drop("C_IND", axis=1)
df_test = df_test.drop("L1YR_C_CNT", axis=1)

In [ ]:
df_test = df_test.replace(["Y", "N"], [1, 0]) 
df_test = df_test.replace(["M", "F"], [1, 0])
df_test = df_test.replace(['low', 'medium high', 'medium', 'high'], [0, 1, 2, 3])
df_test = df_test.replace(['A1', 'A2', 'B1', 'B2', 'D', 'C1', 'C2', 'E'], [0, 1, 2, 3, 4, 5, 6, 7])
df_test = df_test.replace(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], [0, 1, 2, 3, 4, 5, 6, 7])
df_test.head()

### 2. 進行 GLRM 補值

In [ ]:
hf_test = h2o.H2OFrame(df_test)
column_list = hf_test.col_names
for i in column_list:
    if i in categorical_vars:
        hf_test[i] = hf_test[i].asfactor()
    else:
        hf_test[i] = hf_test[i].asnumeric()

In [ ]:
hf_test_hat = glrm_model.predict(hf_test)
hf_test_hat = relu(hf_test_hat)
hf_test_hat

In [ ]:
df_test = hf_test.as_data_frame(use_pandas = True)
df_test_hat = hf_test_hat.as_data_frame(use_pandas = True)

df_test = df_test.astype(np.float32)
df_test_hat = df_test_hat.astype(np.float32)
df_test[df_test.isna()] = df_test_hat[df_test.isna()].values

### 3. 進行 XGBoost 預測

In [ ]:
y_pred = xgbc.predict(df_test)

In [ ]:
df_test["CUS_ID"] = ID
df_test["Ypred"] = y_pred
df_test.head()

In [ ]:
final = df_test[["CUS_ID", "Ypred"]]
final.head()